In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
from datetime import datetime, timedelta
import pandas as pd

# Add project root to Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import src.config as config
from src.inference import get_feature_store, load_model_from_registry, get_model_predictions
from src.data_utils import transform_ts_data_info_features


In [2]:
%load_ext autoreload
%autoreload 2

import sys
import os
from datetime import datetime, timedelta
import pandas as pd

# Add project root to Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import src.config as config
from src.inference import get_feature_store, load_model_from_registry, get_model_predictions
from src.data_utils import transform_ts_data_info_features


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from datetime import timedelta
import pandas as pd

# Define time window
fetch_data_to = pd.Timestamp("2025-04-30 00:00:00", tz="UTC")
fetch_data_from = fetch_data_to - timedelta(days=60)

# Login and load feature store/view
import hopsworks

project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

fs = project.get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION
)

feature_view = fs.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

# Get batch data ±1 day buffer
ts_data = feature_view.get_batch_data(
    start_time=fetch_data_from - timedelta(days=1),
    end_time=fetch_data_to + timedelta(days=1),
)

# Filter strictly for range
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data = ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)


2025-05-10 12:55:14,834 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 12:55:14,843 INFO: Initializing external client
2025-05-10 12:55:14,844 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 12:55:17,316 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.54s) 


In [9]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-03-01 00:00:00,HB101,1
1,2025-03-01 01:00:00,HB101,2
2,2025-03-01 02:00:00,HB101,2
3,2025-03-01 03:00:00,HB101,0
4,2025-03-01 04:00:00,HB101,0
...,...,...,...
7896,2025-04-29 20:00:00,JC115,1
7897,2025-04-29 21:00:00,JC115,4
7898,2025-04-29 22:00:00,JC115,2
7899,2025-04-29 23:00:00,JC115,0


In [10]:
# Create time-series features with sliding window
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)
print("✅ Features ready for prediction:", features.shape)


✅ Features ready for prediction: (172, 674)


In [12]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,1,2,2,0,0,0,1,1,0,1,...,5,11,9,4,2,2,3,4,HB101,2025-03-29 00:00:00
1,4,9,2,2,0,1,0,0,0,0,...,9,8,6,4,7,2,5,0,HB101,2025-03-29 23:00:00
2,0,1,0,0,0,0,0,0,0,1,...,6,5,6,2,2,1,3,2,HB101,2025-03-30 22:00:00
3,4,2,0,0,0,0,0,0,0,0,...,2,2,4,11,12,6,3,1,HB101,2025-03-31 21:00:00
4,1,1,0,0,0,0,0,0,0,0,...,3,0,4,5,8,15,18,3,HB101,2025-04-01 20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,11,5,1,8,5,7,3,5,0,0,...,0,4,8,8,12,15,12,19,JC115,2025-04-25 19:00:00
168,4,7,7,5,6,6,2,2,2,1,...,11,4,4,7,5,2,11,7,JC115,2025-04-26 18:00:00
169,11,2,3,6,8,4,1,7,0,1,...,3,1,7,5,3,9,7,10,JC115,2025-04-27 17:00:00
170,10,27,26,5,5,2,0,0,0,0,...,3,1,2,3,6,5,3,9,JC115,2025-04-28 16:00:00


In [13]:
# Load trained model from registry
model = load_model_from_registry()

# Get predictions
predictions = get_model_predictions(model, features)
print("✅ Predictions generated:", predictions.shape)
predictions.head()


2025-05-10 13:00:52,888 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 13:00:52,898 INFO: Initializing external client
2025-05-10 13:00:52,899 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 13:00:54,163 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
✅ Predictions generated: (172, 2)


,pickup_location_id,predicted_demand
0,HB101,3
1,HB101,2
2,HB101,3
3,HB101,2
4,HB101,2


In [14]:
from src.inference import get_model_predictions

# Step 1: Get predictions
predictions = get_model_predictions(model, features)

# ✅ Step 2: Add pickup_hour back from features
predictions["pickup_hour"] = features["pickup_hour"].values

# Step 3: View top 10 high-demand predictions
top_locations = predictions.sort_values("predicted_demand", ascending=False)
top_locations[["pickup_location_id", "pickup_hour", "predicted_demand"]].head(10)


[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


,pickup_location_id,pickup_hour,predicted_demand
144,JC115,2025-04-03 18:00:00,33
40,HB102,2025-04-03 18:00:00,31
66,HB102,2025-04-28 16:00:00,21
41,HB102,2025-04-04 17:00:00,20
65,HB102,2025-04-27 17:00:00,17
170,JC115,2025-04-28 16:00:00,17
67,HB102,2025-04-29 15:00:00,16
63,HB102,2025-04-25 19:00:00,15
143,JC115,2025-04-02 19:00:00,13
42,HB102,2025-04-05 16:00:00,13
